**DES Block Cipher Implementation**

In [ ]:
#. Note only 56 bits of the original key appear in the permuted key.
PC_1 = [
 57, 49, 41, 33, 25, 17, 9,
 1, 58, 50, 42, 34, 26, 18,
 10, 2, 59, 51, 43, 35, 27,
 19, 11, 3, 60, 52, 44, 36,
 63, 55, 47, 39, 31, 23, 15,
 7, 62, 54, 46, 38, 30, 22,
 14, 6, 61, 53, 45, 37, 29,
 21, 13, 5, 28, 20, 12, 4 ]

#PC-2 only uses 48 of these.
PC_2=[14, 17, 11, 24, 1, 5,
			3, 28, 15, 6, 21, 10,
			23, 19, 12, 4, 26, 8,
			16, 7, 27, 20, 13, 2,
			41, 52, 31, 37, 47, 55,
			30, 40, 51, 45, 33, 48,
			44, 49, 39, 56, 34, 53,
			46, 42, 50, 36, 29, 32]

initial_perm=  [58, 50, 42, 34, 26, 18, 10, 2,
				60, 52, 44, 36, 28, 20, 12, 4,
				62, 54, 46, 38, 30, 22, 14, 6,
				64, 56, 48, 40, 32, 24, 16, 8,
				57, 49, 41, 33, 25, 17, 9, 1,
				59, 51, 43, 35, 27, 19, 11, 3,
				61, 53, 45, 37, 29, 21, 13, 5,
				63, 55, 47, 39, 31, 23, 15, 7]

E_BIT_SELECTION_TABLE=[32, 1, 2, 3, 4, 5, 4, 5,
		                   6, 7, 8, 9, 8, 9, 10, 11,
                        12, 13, 12, 13, 14, 15, 16,
                        17,16, 17, 18, 19, 20, 21, 20, 21,
                        22, 23, 24, 25, 24, 25, 26, 27,
                        28, 29, 28, 29, 30, 31, 32, 1]

# S-box Table
sbox = [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
		[0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
		[4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
		[15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

		[[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
		[3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
		[0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
		[13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

		[[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
		[13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
		[13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
		[1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

		[[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
		[13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
		[10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
		[3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

		[[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
		[14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
		[4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
		[11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

		[[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
		[10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
		[9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
		[4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

		[[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
		[13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
		[1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
		[6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

		[[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
		[1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
		[7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
		[2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]]

#The permutation P is defined in the following table. P yields a 32-bit output from a 32-bit input by permuting the bits of the input block.

P=[16, 7, 20, 21,
	29, 12, 28, 17,
	1, 15, 23, 26,
	5, 18, 31, 10,
	2, 8, 24, 14,
	32, 27, 3, 9,
	19, 13, 30, 6,
	22, 11, 4, 25]

 #apply a final permutation IP-1 as defined by the following table:
final_permutation_IP = [40, 8, 48, 16, 56, 24, 64, 32,
			39, 7, 47, 15, 55, 23, 63, 31,
			38, 6, 46, 14, 54, 22, 62, 30,
			37, 5, 45, 13, 53, 21, 61, 29,
			36, 4, 44, 12, 52, 20, 60, 28,
			35, 3, 43, 11, 51, 19, 59, 27,
			34, 2, 42, 10, 50, 18, 58, 26,
			33, 1, 41, 9, 49, 17, 57, 25]


In [ ]:
#function : convert Hexadeciemal to Binary
def hexTiBinary(s):
  mp={'0':'0000',
      '1':'0001',
      '2':'0010',
      '3':'0011',
      '4':'0100',
      '5':'0101',
      '6':'0110',
      '7':'0111',
      '8':'1000',
      '9':'1001',
      'A':'1010',
      'B':'1011',
      'C':'1100',
      'D':"1101",
      'E':"1110",
      'F':"1111"}
  binary=""
  for i in range(len(s)) :
    binary=binary+mp[s[i]]

  return binary

def binToHex(s):
  mp={'0000':'0',
      '0001':'1',
      '0010':'2',
      '0011': '3',
       '0100':'4',
     '0101': '5',
     '0110': '6',
      '0111':'7',
     '1000': '8',
      '1001':'9',
      '1010':'A',
      '1011':'B',
      '1100':'C',
      "1101":'D',
      "1110":'E',
      "1111":'F'
      }
  hex=""
  for i in range (0,len(s),4):
    chr = ""
    chr = chr+ s[i]
    chr = chr + s[i + 1]
    chr = chr + s[i + 2]
    chr = chr + s[i + 3]
    hex = hex + mp[chr]
  return hex


def binToDeciamal(s):

	binary1 = s
	decimal, i, n = 0, 0, 0
	while(s != 0):
		dec = s % 10
		decimal = decimal + dec * pow(2, i)
		s = s//10
		i += 1
	return decimal



def decToBinary(num):
	res = bin(num).replace("0b", "")
	if(len(res) % 4 != 0):
		div = len(res) / 4
		div = int(div)
		counter = (4 * (div + 1)) - len(res)
		for i in range(0, counter):
			res = '0' + res
	return res


def permute(k, arr, n):
	permutation = ""
	for i in range(0, n):
		permutation = permutation + k[arr[i] - 1]
	return permutation



def shift_left(k, nth_shifts):
	s = ""
	for i in range(nth_shifts):
		for j in range(1, len(k)):
			s = s + k[j]
		s = s + k[0]
		k = s
		s = ""
	return k


def xor(a, b):
		ans = ""
		for i in range(len(a)):
			if a[i] == b[i]:
				ans = ans + "0"
			else:
				ans = ans + "1"
		return ans

def encrypt(m, rkb, rk):
	m = hexTiBinary(m)

	# Initial Permutation
	m = permute(m, initial_perm, 64)
	print("After initial permutation", binToHex(m))

	# splitting into left , right
	left = m[0:32]
	right = m[32:64]
	for i in range(0, 16):
		#  Expanding the 32 bits data into 48 bits
		right_expanded = permute(right, E_BIT_SELECTION_TABLE, 48)
		xor_x = xor(right_expanded, rkb[i])

		sbox_str = ""
		for j in range(0, 8):
			row = binToDeciamal(int(xor_x[j * 6] + xor_x[j * 6 + 5]))
			col = binToDeciamal(
				int(xor_x[j * 6 + 1] + xor_x[j * 6 + 2] + xor_x[j * 6 + 3] + xor_x[j * 6 + 4]))
			val = sbox[j][row][col]
			sbox_str = sbox_str + decToBinary(val)

		sbox_str = permute(sbox_str, P, 32)

		result = xor(left, sbox_str)
		left = result

		if(i != 15):
			left, right = right, left
		print("Round ", i + 1, " ", binToHex(left),
			" ", binToHex(right), " ", rk[i])

	# Combine left+ right
	combine = left + right

	cipher_text = permute(combine, final_permutation_IP, 64)
	return cipher_text


In [ ]:
pt = "0123456789ABCDEF"
key = "133457799BBCDFF1"

# Key generation
key = hexTiBinary(key)


key = permute(key, PC_1, 56)

shift_table = [1, 1, 2, 2,
			2, 2, 2, 2,
			1, 2, 2, 2,
			2, 2, 2, 1]


# Splitting
left = key[0:28]
right = key[28:56]

rkb = []
rk = []
for i in range(0, 16):
	left = shift_left(left, shift_table[i])
	right = shift_left(right, shift_table[i])

	# Combination of left and right string
	combine_str = left + right

	# Compression of key from 56 to 48 bits
	round_key = permute(combine_str, PC_2, 48)

	rkb.append(round_key)
	rk.append(binToHex(round_key))

print("Encryption")
cipher_text = binToHex(encrypt(pt, rkb, rk))
print("Cipher Text : ", cipher_text)


Encryption
After initial permutation CC00CCFFF0AAF0AA
Round  1   F0AAF0AA   EF4A6544   1B02EFFC7072
Round  2   EF4A6544   CC017709   79AED9DBC9E5
Round  3   CC017709   A25C0BF4   55FC8A42CF99
Round  4   A25C0BF4   77220045   72ADD6DB351D
Round  5   77220045   8A4FA637   7CEC07EB53A8
Round  6   8A4FA637   E967CD69   63A53E507B2F
Round  7   E967CD69   064ABA10   EC84B7F618BC
Round  8   064ABA10   D5694B90   F78A3AC13BFB
Round  9   D5694B90   247CC67A   E0DBEBEDE781
Round  10   247CC67A   B7D5D7B2   B1F347BA464F
Round  11   B7D5D7B2   C5783C78   215FD3DED386
Round  12   C5783C78   75BD1858   7571F59467E9
Round  13   75BD1858   18C3155A   97C5D1FABA41
Round  14   18C3155A   C28C960D   5F43B7F2E73A
Round  15   C28C960D   43423234   BF918D3D3F0A
Round  16   0A4CD995   43423234   CB3D8B0E17F5
Cipher Text :  85E813540F0AB405
